In [67]:

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from dataPath import DATA_PATH
%matplotlib inline

import matplotlib

matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [68]:
cap = cv.VideoCapture(DATA_PATH + "videos/video.mp4")
n_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

fps = int(cap.get(cv.CAP_PROP_FPS))

out = cv.VideoWriter("video_out.avi", cv.VideoWriter_fourcc("M", "J", "P", "G"), fps, ((w * 2), h))

SMOOTHING_RADIUS = 50

In [69]:
_, prev = cap.read()
prev_gray = cv.cvtColor(prev, cv.COLOR_BGR2GRAY)

In [70]:
transforms = np.zeros((n_frames-1,3),np.float32)

for i in range(n_frames - 2):
    prev_pts = cv.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

    success, curr = cap.read()

    if not success:
        break

    curr_gray = cv.cvtColor(curr,cv.COLOR_BGR2GRAY)
    curr_pts, status, err = cv.calcOpticalFlowPyrLK(prev_gray,curr_gray,prev_pts,None)

    assert prev_pts.shape == curr_pts.shape

    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]

    m = cv.estimateAffinePartial2D(prev_pts,curr_pts)

    # Extract traslation
    dx = m[0][0,2]
    dy = m[0][1,2]

    # Extract rotation angle
    da = np.arctan2(m[0][1,0], m[0][0,0])

    # Store transformation
    transforms[i] = [dx,dy,da]

    # Move to next frame
    prev_gray = curr_gray

    #print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

In [71]:
#trajectory = np.cumsum(transforms,axis=0)

In [72]:
def movingAverage(curve,radius):
    window_size = 2 * radius + 1
    f = np.ones(window_size)/window_size
    curve_pad = np.lib.pad(curve,(radius,radius),'edge')
    curve_smoothed = np.convolve(curve_pad,f, mode= 'same')
    curve_smoothed = curve_smoothed[radius:-radius]
    return curve_smoothed

In [73]:
def smooth(trajectory):
    smoothed_trajectory = np.copy(trajectory)

    for i in range(3):
        smoothed_trajectory[:,i]=movingAverage(trajectory[:,i],radius=SMOOTHING_RADIUS)

    return smoothed_trajectory

In [74]:
trajectory = np.cumsum(transforms,axis=0)

smooth_trajectory = smooth(trajectory)

difference = smooth_trajectory - trajectory
transforms_smooth = transforms + difference

In [75]:
def fixBorder(frame):
    s = frame.shape
    T = cv.getRotationMatrix2D((s[1]/2,s[0]/2),0,1.04)
    frame = cv.warpAffine(frame,T,(s[1],s[0]))
    return frame

In [76]:
cap.set(cv.CAP_PROP_POS_FRAMES,0)

True

In [77]:
for i in range(n_frames-2):
    success, frame = cap.read()

    if not success:
        break

    dx = transforms_smooth[i,0]
    dy = transforms_smooth[i,1]
    da = transforms_smooth[i,2]

    m = np.zeros((2,3),np.float32)
    m[0,0] = np.cos(da)
    m[0,1] = -np.sin(da)
    m[1,0] = np.sin(da)
    m[1,1] = np.cos(da)
    m[0,2] = dx
    m[1,2] = dy

    frame_stabilized = cv.warpAffine(frame,m,(w,h))

    frame_stabilized = fixBorder(frame_stabilized)

    # Write the frame to the file
    frame_out = cv.hconcat([frame, frame_stabilized])

    # If the image is too big, resize it.
    if(frame_out.shape[1] > 1920):
        frame_out = cv.resize(frame_out, (w,h))
    #cv2.imshow("Frame",frame_out)
    #cv2.waitKey(0)

    out.write(frame_out)

In [78]:
cv.destroyAllWindows()
out.release()